In [1]:
!pip install konlpy

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 562 kB/s 
     |████████████████████████████████| 453 kB 33.1 MB/s 
Mounted at /content/drive


In [2]:
import random
import scipy as sp 
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Hannanum
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
dic = {'tv': [62],   # 텔레비전, 티비
 '가방': [24, 26],
 '가위': [76],  # 문구
 '간식': [52, 53, 54, 55],
 '강아지': [16], # 견종, 멍멍이, 멍멍
 '개': [16],
 '거실': [57],
 '경마': [17],
 '고속열차': [6],
 '고양이': [15],    # 야옹이, 야옹
 '곰': [21],
 '곰돌이': [21],
 '곰인형': [77],
 '공': [32],
 '공항': [4],
 '과일': [46, 47, 49],
 '교통': [9],
 '그릇': [45],
 '글라스': [40],
 '기린': [23],
 '기차': [6],
 '까치': [14],
 '꽃': [58, 75],
 '꽃병': [75],
 '나이프': [43],
 '냉장고': [72],
 '넥타이': [27],
 '노트북': [63],
 '당근': [51],
 '대중교통': [5],
 '도넛': [54],
 '도로': [9, 81],
 '도서관': [73],
 '독서': [73],
 '동물': [15, 16, 17, 18, 19, 20, 21, 22, 23],
 '드라이기': [78],
 '디저트': [54, 55],
 '라켓': [38],
 '랩탑': [63],
 '리모컨': [65],
 '리모콘': [65],
 '마우스': [64],
 '말': [17],
 '목장': [19],
 '물병': [39],
 '물컵': [41],
 '미니백': [26],
 '바나나': [46],
 '밥그릇': [45],
 '방망이': [34],
 '배': [8],
 '배낭': [24],
 '배달': [3],
 '배트': [34],
 '백': [26],
 '백팩': [24],
 '버스': [5],
 '벤치': [13],
 '변기': [61],
 '변기통': [61],
 '병': [39],
 '병류': [39],
 '보드': [31, 36],
 '보트': [8],
 '볼': [32, 45],
 '부엌': [71, 72],
 '북극곰': [21],
 '브로콜리': [50],
 '비둘기': [14],
 '비행기': [4],
 '빵': [48, 54, 55],
 '사과': [47],
 '사람': [0],
 '사람들': [0],
 '새': [14],
 '샌드위치': [48],
 '생일': [55],
 '서프': [37],
 '서프보드': [37],
 '서핑': [37],
 '세면대': [61],
 '소': [19],
 '소떼': [19],
 '소파': [57],
 '소화기': [10],
 '소화전': [10],
 '숄더백': [26],
 '수저': [44],
 '숟가락': [44],
 '술병': [39],
 '술잔': [40],
 '스노우보드': [31],
 
 '스케이트': [36],
 '스케이트보드': [36],
 '스키': [30],
 '스키보드': [30],
 '스키장': [30, 31],
 '스포츠공': [32],
 '스푼': [44],
 '승마': [17],
 '승마장': [17],
 '승용차': [2],
 '시계': [74],
 '식기류': [42],
 '식사': [40, 41, 42, 43, 44, 45, 48, 52, 53, 60],
 '식탁': [60],
 '신호등': [9],
 '싱크': [71],
 '싱크대': [71],
 '안방': [59],
 '야구': [34, 34, 35],
 '야구방망이': [34],
 '야구장갑': [35],
 '야채': [50, 51],
 '양': [18],
 '양떼': [18],
 '양치': [79],
 '얼룩말': [22],
 '여행': [28],
 '여행가방': [28],
 '연': [33],
 '연날리기': [33],
 '열차': [6],
 '오렌지': [49],
 '오븐': [69],
 '오토바이': [3],
 '와인잔': [40],
 '요리': [50, 51, 68, 69],
 '요트': [8],
 '우산': [25],
 '운동': [32, 34, 35, 36, 37],
 '운전': [2],
 '원반': [29],
 '원반던지기': [29],
 '유리컵': [41],
 '음메': [19],
 '음식': [45, 48, 52, 53, 54, 55],
 '의자': [13, 56],
 '인형': [77],
 '일': [63],
 '자동차': [2],
 '자전거': [1],
 '작업': [63],
 '잠': [59],
 '장갑': [35],
 '전동차': [3],
 '전자레인지': [68],
 '전화' : [67],
 '접시': [45],
 '정지': [11],
 '정지 신호': [11],
 '조류': [14],
 '주스병': [39],
 '주차': [12],
 '주차 미터기': [12],
 '주차장': [12],
 '차': [2],
 '차량': [2],
 '참새': [14],
 '채소': [50, 51],
 '책': [73],
 '책방': [73],
 '책장': [73],
 '출국': [28],
 '치약': [79],
 '침대': [59],
 '침실': [59],
 '칫솔': [79],
 '칼': [43],
 '캐리어': [28],
 '컴퓨터': [63],
 '컵': [41],
 '케이크': [55],
 '코끼리': [20],
 '키보드': [66],
 '타이': [27],
 '타자': [34],
 '테니스': [38],
 '테니스 라켓': [38],
 '테디베어': [77],
 '토스터': [70],
 '토스트': [48, 70],
 '토스트기': [70],
 '투수': [35],
 '트럭': [7],
 '포수': [35],
 '포크': [42],
 '핸드폰': [67], #폰, #휴대폰
 '스마트폰': [67],

 '프리스비': [29],
 '프리즈비': [29],
 '피자': [53],
 '핫도그': [52],
 '항공': [4],
 '해양스포츠': [37],
 '핸드백': [26],
 
 '화병': [75],
 '화분': [58],
 '화장실': [61, 79],
 '화초': [58],
 


 

 '벨소리' : [80],
 '알람소리' : [80],
 '경적소리' : [81],
 '박수소리' : [82],
 '아기울음소리' : [83],
 '드럼소리' : [84],
 '기타소리' : [85],
 '피아노소리' : [86],
 '악기소리' : [84, 85, 86],
 '사자소리' : [87],
 '총소리' : [88],
 '빗소리' : [89],
 '비명소리' : [90],
 '사이렌소리' : [91],
 '코고는소리' : [92],
 '웃음소리' : [93],
 '천둥소리' : [94],
 '청소기소리' : [95],


 '비행기소리' : [96],
 '새소리' : [97],
 '강아지소리' : [98],
 '고양이소리' : [99],
 
 


#  'others' : [1000],
 
 
 }

In [4]:
def norm_cos_sim(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    delta = cosine_similarity(v1_normalized, v2_normalized)
    return sp.linalg.norm(delta) 
    # return sp.linalg.norm(delta.toarray())

class Scene:

    def __init__(self, od_raw, sd_raw, o_url, y_url, n=2 ):
        self.od_raw = od_raw
        self.sd_raw = sd_raw
        self.o_url = o_url
        self.y_url = y_url
    
    def analysis(self, n=2):
        # n = 자르는 초단위
        detector = []
        for time in tqdm(range(0, int(self.od_raw.iloc[-1,1] + n), n)):
        # for time in tqdm(range(0, 100, n)):
            od_tmp = []
            for i in range(len(self.od_raw)) :
                if (time < self.od_raw.iloc[i,0] <= time+n) or (time < self.od_raw.iloc[i,1] <= time+n):
                    dect_num = int(self.od_raw.iloc[i,2].split('_')[0])
                    for key, val in dic.items():
                        # print(key, val, dect_num)
                        if val.count(dect_num) != 0:
                            # print(f'{time} ~ {time+3} : {key}')
                            od_tmp.append(key)
            if od_tmp == []:
                continue
            else:
                detector.append({
                "start" : time,
                "end" : time+n,
                "detect" : ' '.join(list(set(od_tmp)))
                })



        df_detector = pd.DataFrame(detector)

        for time in tqdm(range(0, int(self.sd_raw.iloc[-1,1] + n), n)):
        # for time in tqdm(range(0, 100, 3)):
            sd_tmp = []
            for i in range(len(self.sd_raw)) :
                if (time < self.sd_raw.iloc[i,0] <= time+n) or (time < self.sd_raw.iloc[i,1] <= time+n):
                    dect_num = self.sd_raw.iloc[i,2]
                    for key, val in dic.items():
                        # print(key, val, dect_num)
                        if val.count(dect_num) != 0:
                            # print(f'{time} ~ {time+3} : {key}')
                            sd_tmp.append(key)
            if sd_tmp == []:
                continue
            else:
                sd_tmp = ' '.join(list(set(sd_tmp)))
                try:
                    df_detector.iloc[int(((time+n)/2)-1), 2] += ' ' + sd_tmp
                except:
                    pass

        contents_for_vectorize = []
        for idx, row in df_detector.iterrows():
            contents_for_vectorize.append(row[2])

        stop_words = ['소리', '장면']
        vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore', stop_words=stop_words)
        self.X = vectorizer.fit_transform(contents_for_vectorize)
        self.df_detector = df_detector
        self.contents_for_vectorize = contents_for_vectorize
        self.vectorizer = vectorizer

    def search(self):
        hannanum = Hannanum()
        query = input('검색어를 입력하세요')
        tokens = [token for token in hannanum.nouns(query) if token != '장면' and '소리']
        new_post_tokens = []

        for token in tokens:
            tmp = []
            for key, val in dic.items():
                try:
                    if token == key:
                        dect_num = val

                        for n in dect_num :
                            for key2, val2 in dic.items():
                                if val2.count(n) != 0:
                                    tmp.append(key2)
                except:
                    pass

            if tmp == []:
                continue
            else:
                for i in range(len(tmp)):
                    new_post_tokens.append(tmp[i])


        new_post_tokens = list(set(new_post_tokens))
        new_post_for_vectorize = [' '.join(new_post_tokens)]
        new_post_vec = self.vectorizer.transform(new_post_for_vectorize)


        dist = [norm_cos_sim(each, new_post_vec) for each in self.X]

        dist_rank = [(val, idx) for idx, val in enumerate(dist)]
        dist_rank.sort(reverse=True)

        print(f'input query ==> "{query}"')
        print(f'input query의 토큰화 ==> {new_post_for_vectorize}')
        print('-'*50)
        print(f'Best post is {dist.index(max(dist))}, dist = {max(dist)}')
        print('-'*50)
        print()
        print(f"""[TOP 5] First is  ==> {self.df_detector['start'][dist_rank[0][1]]}초 ~ {self.df_detector['end'][dist_rank[0][1]]}초 || {self.contents_for_vectorize[dist_rank[0][1]]}
                origin  : {self.o_url}{int(self.df_detector['start'][dist_rank[0][1]])}s
                yolo    : {self.y_url}{int(self.df_detector['start'][dist_rank[0][1]])}s""");
        print(f"""[TOP 5] Second is ==> {self.df_detector['start'][dist_rank[1][1]]}초 ~ {self.df_detector['end'][dist_rank[1][1]]}초 || {self.contents_for_vectorize[dist_rank[1][1]]}
                origin  : {self.o_url}{int(self.df_detector['start'][dist_rank[1][1]])}s
                yolo    : {self.y_url}{int(self.df_detector['start'][dist_rank[1][1]])}s""");
        print(f"""[TOP 5] Third is  ==> {self.df_detector['start'][dist_rank[2][1]]}초 ~ {self.df_detector['end'][dist_rank[2][1]]}초 || {self.contents_for_vectorize[dist_rank[2][1]]}
                origin  : {self.o_url}{int(self.df_detector['start'][dist_rank[2][1]])}s
                yolo    : {self.y_url}{int(self.df_detector['start'][dist_rank[2][1]])}s""");
        print(f"""[TOP 5] Fourth is ==> {self.df_detector['start'][dist_rank[3][1]]}초 ~ {self.df_detector['end'][dist_rank[3][1]]}초 || {self.contents_for_vectorize[dist_rank[3][1]]}
                origin  : {self.o_url}{int(self.df_detector['start'][dist_rank[3][1]])}s
                yolo    : {self.y_url}{int(self.df_detector['start'][dist_rank[3][1]])}s""");
        print(f"""[TOP 5] Fifth is  ==> {self.df_detector['start'][dist_rank[4][1]]}초 ~ {self.df_detector['end'][dist_rank[4][1]]}초 || {self.contents_for_vectorize[dist_rank[4][1]]}
                origin  : {self.o_url}{int(self.df_detector['start'][dist_rank[4][1]])}s
                yolo    : {self.y_url}{int(self.df_detector['start'][dist_rank[4][1]])}s""");
    

---

# Local data

In [ ]:
# 플란다스의 개
f_i_df = pd.read_csv('./flanders_i.csv', header=None, index_col=False, names=['drop', 'start', 'end', 'object_nums', 4, 5, 6, 7])
f_i_df.drop(['drop'], axis=1, inplace=True)
f_s_df = pd.read_csv('./flanders_s.csv', index_col=0)

# 그것만이 내세상
g_i_df = pd.read_csv('./sesang_i.csv', header=None, index_col=False, names=['drop', 'start', 'end', 'object_nums', 4, 5, 6, 7])
g_i_df.drop(['drop'], axis=1, inplace=True)
g_s_df = pd.read_csv('./sesang_s.csv', index_col=0)

C:\Users\user\AppData\Local\Temp/ipykernel_32240/85940440.py:2: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  f_i_df = pd.read_csv('./flanders_i.csv', header=None, index_col=False, names=['drop', 'start', 'end', 'object_nums', 4, 5, 6, 7])
C:\Users\user\AppData\Local\Temp/ipykernel_32240/85940440.py:7: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  g_i_df = pd.read_csv('./sesang_i.csv', header=None, index_col=False, names=['drop', 'start', 'end', 'object_nums', 4, 5, 6, 7])


---

# Colab data

In [5]:
# 플란다스의 개
f_i_df = pd.read_csv('/content/drive/MyDrive/DRIVE_ds_study/Vmark/flanders_i.csv', header=None, index_col=False, names=['drop', 'start', 'end', 'object_nums', 4, 5, 6, 7])
f_i_df.drop(['drop'], axis=1, inplace=True)
f_s_df = pd.read_csv('/content/drive/MyDrive/DRIVE_ds_study/Vmark/flanders_s.csv', index_col=0)

# 그것만이 내세상
g_i_df = pd.read_csv('/content/drive/MyDrive/DRIVE_ds_study/Vmark/sesang_i.csv', header=None, index_col=False, names=['drop', 'start', 'end', 'object_nums', 4, 5, 6, 7])
g_i_df.drop(['drop'], axis=1, inplace=True)
g_s_df = pd.read_csv('/content/drive/MyDrive/DRIVE_ds_study/Vmark/sesang_s.csv', index_col=0)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


---

# URL

In [6]:
url_dict = {"flanders_original": "https://www.youtube.com/watch?v=yMV4lX8qaV8&t=",
            "flanders_yolo" :  "https://www.youtube.com/watch?v=bD9GGdA6uGQ&t=",
            "sesang_original": "https://www.youtube.com/watch?v=try6rRGsdrw&t=",
            "sesang_yolo" :  "https://www.youtube.com/watch?v=VskOZglJiio&t=",}

In [ ]:
#@title
def scene_search(od_raw, sd_raw, o_url, y_url, n=2):
    # n = 자르는 초단위
    detector = []
    for time in tqdm(range(0, int(od_raw.iloc[-1,1] + n), n)):
    # for time in tqdm(range(0, 100, n)):
        od_tmp = []
        for i in range(len(od_raw)) :
            if (time < od_raw.iloc[i,0] <= time+n) or (time < od_raw.iloc[i,1] <= time+n):
                dect_num = int(od_raw.iloc[i,2].split('_')[0])
                for key, val in dic.items():
                    # print(key, val, dect_num)
                    if val.count(dect_num) != 0:
                        # print(f'{time} ~ {time+3} : {key}')
                        od_tmp.append(key)
        if od_tmp == []:
            continue
        else:
            detector.append({
            "start" : time,
            "end" : time+n,
            "detect" : ' '.join(list(set(od_tmp)))
            })



    df_detector = pd.DataFrame(detector)

    for time in tqdm(range(0, int(sd_raw.iloc[-1,1] + n), n)):
    # for time in tqdm(range(0, 100, 3)):
        sd_tmp = []
        for i in range(len(sd_raw)) :
            if (time < sd_raw.iloc[i,0] <= time+n) or (time < sd_raw.iloc[i,1] <= time+n):
                dect_num = sd_raw.iloc[i,2]
                for key, val in dic.items():
                    # print(key, val, dect_num)
                    if val.count(dect_num) != 0:
                        # print(f'{time} ~ {time+3} : {key}')
                        sd_tmp.append(key)
        if sd_tmp == []:
            continue
        else:
            sd_tmp = ' '.join(list(set(sd_tmp)))
            try:
                df_detector.iloc[int(((time+n)/2)-1), 2] += ' ' + sd_tmp
            except:
                pass

    contents_for_vectorize = []
    for idx, row in df_detector.iterrows():
        contents_for_vectorize.append(row[2])

    stop_words = ['소리', '장면']
    vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore', stop_words=stop_words)
    X = vectorizer.fit_transform(contents_for_vectorize)

    hannanum = Hannanum()
    query = input('검색어를 입력하세요')
    tokens = [token for token in hannanum.nouns(query) if token != '장면' and '소리']
    new_post_tokens = []

    for token in tokens:
        tmp = []
        for key, val in dic.items():
            try:
                if token == key:
                    dect_num = val

                    for n in dect_num :
                        for key2, val2 in dic.items():
                            if val2.count(n) != 0:
                                tmp.append(key2)
            except:
                pass

        if tmp == []:
            continue
        else:
            for i in range(len(tmp)):
                new_post_tokens.append(tmp[i])


    new_post_tokens = list(set(new_post_tokens))
    new_post_for_vectorize = [' '.join(new_post_tokens)]
    new_post_vec = vectorizer.transform(new_post_for_vectorize)


    dist = [norm_cos_sim(each, new_post_vec) for each in X]

    dist_rank = [(val, idx) for idx, val in enumerate(dist)]
    dist_rank.sort(reverse=True)

    print(f'input query ==> "{query}"')
    print(f'input query의 토큰화 ==> {new_post_for_vectorize}')
    print('-'*50)
    print(f'Best post is {dist.index(max(dist))}, dist = {max(dist)}')
    print('-'*50)
    print()
    print(f"""[TOP 5] First is  ==> {df_detector['start'][dist_rank[0][1]]}초 ~ {df_detector['end'][dist_rank[0][1]]}초 || {contents_for_vectorize[dist_rank[0][1]]}
            origin  : {o_url}{int(df_detector['start'][dist_rank[0][1]])}s
            yolo    : {y_url}{int(df_detector['start'][dist_rank[0][1]])}s""");
    print(f"""[TOP 5] Second is ==> {df_detector['start'][dist_rank[1][1]]}초 ~ {df_detector['end'][dist_rank[1][1]]}초 || {contents_for_vectorize[dist_rank[1][1]]}
            origin  : {o_url}{int(df_detector['start'][dist_rank[1][1]])}s
            yolo    : {y_url}{int(df_detector['start'][dist_rank[1][1]])}s""");
    print(f"""[TOP 5] Third is  ==> {df_detector['start'][dist_rank[2][1]]}초 ~ {df_detector['end'][dist_rank[2][1]]}초 || {contents_for_vectorize[dist_rank[2][1]]}
            origin  : {o_url}{int(df_detector['start'][dist_rank[2][1]])}s
            yolo    : {y_url}{int(df_detector['start'][dist_rank[2][1]])}s""");
    print(f"""[TOP 5] Fourth is ==> {df_detector['start'][dist_rank[3][1]]}초 ~ {df_detector['end'][dist_rank[3][1]]}초 || {contents_for_vectorize[dist_rank[3][1]]}
            origin  : {o_url}{int(df_detector['start'][dist_rank[3][1]])}s
            yolo    : {y_url}{int(df_detector['start'][dist_rank[3][1]])}s""");
    print(f"""[TOP 5] Fifth is  ==> {df_detector['start'][dist_rank[4][1]]}초 ~ {df_detector['end'][dist_rank[4][1]]}초 || {contents_for_vectorize[dist_rank[4][1]]}
            origin  : {o_url}{int(df_detector['start'][dist_rank[4][1]])}s
            yolo    : {y_url}{int(df_detector['start'][dist_rank[4][1]])}s""");

---

# [플란다스의 개]

### (장면분석)

In [13]:
flanders = Scene(f_i_df, f_s_df, url_dict["flanders_original"], url_dict["flanders_yolo"])

In [14]:
flanders.analysis()

100%|██████████| 270/270 [00:10<00:00, 26.59it/s]


### (장면 검색)

In [20]:
flanders.search()

검색어를 입력하세요비행기가 나오는 장면
input query ==> "비행기가 나오는 장면"
input query의 토큰화 ==> ['비행기 항공 공항']
--------------------------------------------------
Best post is 107, dist = 0.7150053274933308
--------------------------------------------------

[TOP 5] First is  ==> 368초 ~ 370초 || 비둘기 조류 항공 공항 참새 비행기 까치 새 강아지소리
                origin  : https://www.youtube.com/watch?v=yMV4lX8qaV8&t=368s
                yolo    : https://www.youtube.com/watch?v=bD9GGdA6uGQ&t=368s
[TOP 5] Second is ==> 218초 ~ 220초 || 항공 사람 고속열차 공항 비행기 사람들 소화전 소화기 열차 기차 강아지소리
                origin  : https://www.youtube.com/watch?v=yMV4lX8qaV8&t=218s
                yolo    : https://www.youtube.com/watch?v=bD9GGdA6uGQ&t=218s
[TOP 5] Third is  ==> 540초 ~ 542초 || 사람 사람들 악기소리 피아노소리
                origin  : https://www.youtube.com/watch?v=yMV4lX8qaV8&t=540s
                yolo    : https://www.youtube.com/watch?v=bD9GGdA6uGQ&t=540s
[TOP 5] Fourth is ==> 538초 ~ 540초 || 사람 사람들 악기소리 피아노소리
                origin  : https://www.youtube.c

# [그것만이 내 세상] : 저작권 문제로 시연은 불가

In [ ]:
sesang = Scene(g_i_df, g_s_df, url_dict["sesang_original"], url_dict["sesang_yolo"])

### 장면 분석

In [ ]:
sesang.analysis()

100%|██████████| 418/418 [00:18<00:00, 22.69it/s]


### 장면 검색

In [ ]:
sesang.search()

input query ==> "강아지소리가 들리는 장면"
input query의 토큰화 ==> ['강아지소리']
--------------------------------------------------
Best post is 6, dist = 0.6287296308418321
--------------------------------------------------

[TOP 5] First is  ==> 768초 ~ 770초 || 사람들 사람 강아지소리
                origin  : https://www.youtube.com/watch?v=try6rRGsdrw&t=768s
                yolo    : https://www.youtube.com/watch?v=VskOZglJiio&t=768s
[TOP 5] Second is ==> 722초 ~ 724초 || 사람들 사람 강아지소리
                origin  : https://www.youtube.com/watch?v=try6rRGsdrw&t=722s
                yolo    : https://www.youtube.com/watch?v=VskOZglJiio&t=722s
[TOP 5] Third is  ==> 676초 ~ 678초 || 사람들 사람 강아지소리
                origin  : https://www.youtube.com/watch?v=try6rRGsdrw&t=676s
                yolo    : https://www.youtube.com/watch?v=VskOZglJiio&t=676s
[TOP 5] Fourth is ==> 674초 ~ 676초 || 사람들 사람 강아지소리
                origin  : https://www.youtube.com/watch?v=try6rRGsdrw&t=674s
                yolo    : https://www.youtube.com/wat